# Evolutionary Programming and Differential Evolution Algorithms [20 marks]

In this question, you have the task to implement 
1. the Evolutionary Programming (EP) algorithm and 
2. either the Differential Evolution (DE) or Evolution Strategy (ES) algorithms. 

You also need to apply the implemented algorithms to searching for the minimum of the following two functions, where D is the number of variables, i.e., x1, x2, ..., xD

These two functions are the Rosenbrock's function and the Griewank's function.

For D = 20, d0o the following :
- Implement any specific variation of the EP and DE/ES algorithms of your choice. Justify your choice of the EP and DE/ES algorithm variations implemented.
- Choose appropriate algorithm parameters and population size, in line with your algorithm implementations.
- Determine the fitness function, solution encoding, and stopping criterion in EP and DE/ES.
- Since EP and DE/ES are stochastic algorithms, for each function, f1(x) or f2(x), repeat the experiments 30 times, and report the mean and standard deviation of your results,
i.e., f1(x) or f2(x) values.
- Analyze your results, and draw your conclusions

In [1]:

import copy
import math
import random
import time  # for time measurement
from copy import deepcopy
from random import sample

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

define some hyperparameters 

meanins of each hyperparameter are inspired by the 
1. slide 
2. https://towardsdatascience.com/unit-5-evolutionary-programming-cced3a00166a
3. https://machinelearningmastery.com/evolution-strategies-from-scratch-in-python/
4. `https://blog.csdn.net/qq_40434430/article/details/88745128?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522166593594516782417023563%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=166593594516782417023563&biz_id=0&utm_medium=distribute.pc_chrome_plugin_search_result.none-task-blog-2~all~sobaiduend~default-1-88745128-null-null.nonecase&utm_term=fast%20evolutionary%20programming&spm=1018.2226.3001.4187#t5`

In [17]:
# np.random.seed(1)
MAX_GENS = 700
POPULATION_SIZE = 250
# mutation argunments to control the mutation strength 
t1 = (1/(2*POPULATION_SIZE))**0.5   
t2 = (1/(2*(POPULATION_SIZE**0.5)))**0.5   
mu = 3.0 # initial sigma for Fast EP

mut_strength_step_size = 0.15 # define the maximum step size for ES (1+1)

# following hyperparameters are for ES, but due to 1+1 ES is implemented, so they are actually not used
mu_es = 20 #  selected parents
lam = 100 # number of offspring generated by parents
n_children = int(lam/mu_es) 


In [18]:
def Rosenbrock_function(X):
    # evaluate the Rosenbrock function
    # input:
    #   X: the input vector
    # output:
    #   the value of the Rosenbrock function

    return np.sum(100 * (X[1:] - X[:-1] ** 2) ** 2 + (1 - X[:-1]) ** 2) 
                # - np.prod(np.cos(X/np.sqrt(np.arange(1,X.size+1))))


def Griewanks_function(X):
    """Evaluate the griewanks function

    Args:
        X (numpy.ndarray): the input vector
        
    Returns:
        float: the value of the griewanks function
    """
    return 1+1/4000*np.sum(X**2) \
            - np.prod(np.cos(X/np.sqrt(np.arange(1,X.size+1))))

    # return 1+ (X[0] ** 2 + X[1] ** 2) - np.cos(X[0]) * np.cos(X[1] / np.sqrt(2)) 

Define the Individual for EP.

The mutation code is inspired from 

1. [csdn](https://blog.csdn.net/qq_40434430/article/details/88745128?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522166593594516782417023563%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=166593594516782417023563&biz_id=0&utm_medium=distribute.pc_chrome_plugin_search_result.none-task-blog-2~all~sobaiduend~default-1-88745128-null-null.nonecase&utm_term=fast%20evolutionary%20programming&spm=1018.2226.3001.4187#t7)


In [20]:
class IndividualEP:
    """
    EP is about behaviour evolution, not about the evolution of the genotype.
    Due to the genotype is not evolved, no crossover method is used between parents, only mutation. 
    Individuals are created only by mutating the parents, where the amount of mutation is known as the behavior.
    """
    def __init__(self, D, evaluatorFunction , x_min=-30, x_max=30):
        self.x_min = x_min
        self.x_max = x_max
        self.D = D   # dimensions
        self.evaluatorFunction = evaluatorFunction # the function to be optimized, e.g. Rosenbrock_function
        self.vector = np.random.uniform(x_min, x_max, D)
        self.variance = np.array([mu] * D)
        self.fitness = self.evaluate()
        
    def evaluate(self):
        # check if the individual is within the search space
        if np.any(self.vector < self.x_min) or np.any(self.vector > self.x_max):
            return np.inf # inf means invalid individual, which will be removed from the population
        
        self.fitness = self.evaluatorFunction(self.vector)
        return self.fitness
        
    @staticmethod 
    def mutation(population):
        """Mutate the population, due to it is the fast EP, we use the cauchy distribution.
        
        Also, the mutation is both applied on the vector and the variance.

        Args:
            population (list): the population to be mutated

        Returns:
            list: the mutated population, i.e. offspring
        """
        pop1 = deepcopy(population)
        for ind in pop1:
            dna, sigma = ind.vector, ind.variance
            temp = np.random.randn()
            temp1 = np.random.randn(*sigma.shape)
            sigma = sigma * np.exp(t1 * temp + t2 * temp1)
            dna = dna + sigma * np.random.standard_cauchy(dna.shape)
            # avoid out of bound
            dna = np.clip(dna, ind.x_min, ind.x_max)
            
            ind.vector = dna
            ind.variance = sigma
            ind.evaluate()
        return pop1
            
        # offspring = []
        # for individual in population :
        #     mutated_individual = deepcopy(individual)
        #     # mutate the vector by adding a cauchy distributed random number
        #     # eta: the amount of mutation, i.e. the behavior
        #     # delta: the variance of the cauchy distribution
        #     mutated_individual.vector += np.random.standard_cauchy(mutated_individual.D) * mutated_individual.variance  # eta * delta
            
        #     # then, mutate the variance 
        #     tau = 1/np.sqrt(2*mutated_individual.D)
        #     tau_prime = 1/np.sqrt(2*np.sqrt(mutated_individual.D))
        #     mutated_individual.variance *= np.exp(
        #         tau * np.random.standard_cauchy(mutated_individual.D) 
        #         + tau_prime * np.random.standard_cauchy(mutated_individual.D)
        #         )
            
        #     # clip the vector to be in the range of [x_min, x_max]
        #     mutated_individual.vector = np.clip(mutated_individual.vector, mutated_individual.x_min, mutated_individual.x_max)
        #     # # clip the variance to be in the range of [0.1, 10]
        #     # mutated_individual.variance = np.clip(mutated_individual.variance, 0.1, 10)
        #     # evaluate the fitness
        #     mutated_individual.fitness = mutated_individual.evaluate()
            
        #     offspring.append(mutated_individual)
        # return offspring
            
        
    @staticmethod
    def elitism_selection(parents, offspring):
        """Select the best individuals from the parents and the offspring.

        Args:
            parents (list): the parents
            offspring (list): the offspring

        Returns:
            list: the selected individuals
        """
        return sorted(parents + offspring, key=lambda x: x.fitness)[0:POPULATION_SIZE]
    
    @staticmethod
    def tournament_selection(parents, offspring):
        pool = parents + offspring
        pool = deepcopy(pool) # to avoid the mutation of the original pool
        selected = []
        # we are using 10 individuals for the tournament 
        while len(selected) < POPULATION_SIZE:
            tournament = sample(pool, 10) # 10 individuals for the tournament
            selected.append(min(tournament, key=lambda x: x.fitness))
            # remove the selected individual from the pool
            pool.remove(selected[-1])
        # sort the selected individuals by their fitness
        selected.sort(key=lambda x: x.fitness)
        
        return selected
    
    
    def __repr__(self):
        return f"IndividualEP(vector={self.vector}, fitness={self.fitness})"
    
    def __str__(self):
        return f"IndividualEP(vector={self.vector}, fitness={self.fitness})"
    
    def __eq__(self, other):
        return self.fitness == other.fitness
    
    


Then, define the Individual class for Evolution Strategy (ES) algorithm.

https://github.com/MorvanZhou/Evolutionary-Algorithm/blob/master/tutorial-contents/Evolution%20Strategy/(1%2B1)-ES.py

https://machinelearningmastery.com/evolution-strategies-from-scratch-in-python/

In [21]:
class IndividialES(IndividualEP):
    """_summary_

    Args:
        IndividualEP (_type_): _description_
    """
    
    def __init__(self, D, evaluatorFunction , x_min=-30, x_max=30):
        super().__init__(D, evaluatorFunction, x_min, x_max)
    
    
    @staticmethod
    def make_offspring(parent):
        offspring = deepcopy(parent)
        offspring.vector += np.random.randn(*offspring.vector.shape) * mut_strength_step_size
        offspring.vector = np.clip(offspring.vector, offspring.x_min, offspring.x_max)
        offspring.fitness = offspring.evaluate()
        return offspring
    
    @staticmethod
    def select_best(parent, offspring):
        global mut_strength_step_size
        p_target = 0.2
        ps = 0. if parent.fitness < offspring.fitness else 1
        # adjust the step size
        mut_strength_step_size *=  np.exp(1/np.sqrt(offspring.D+1) * (ps - p_target)/(1 - p_target))
        return parent if parent.fitness < offspring.fitness else offspring
        
        
    

Run the Fast EP to solve Rosenbrock function and Griewanks function
Fast EP uses the Cauchy distribution for mutating the genome with a scale parameters of 1, along with lognormal adaption for updating the strategy parameter, and elitism for selection for survival. 


In [22]:
def fast_EP(population_size, D, evaluatorFunction, x_min=-30, x_max=30):
    """The fast EP algorithm.

    Args:
        population_size (int): the size of the population
        D (int): the dimension of the problem
        evaluatorFunction (function): the function to be optimized, e.g. Rosenbrock_function
        x_min (int, optional): the minimum value of the vector. Defaults to -30.
        x_max (int, optional): the maximum value of the vector. Defaults to 30.

    Returns:
        list: the best individual
    """
    # create the initial population
    population = [IndividualEP(D, evaluatorFunction, x_min, x_max) for _ in range(population_size)]
    # the fitness is already evaluated when IndividualEP is created
    #  so we don't need to evaluate the fitness again
    
    for _ in range(MAX_GENS):
        # mutate the population
        offspring = IndividualEP.mutation(population)
        # select the best individuals from the parents and the offspring
        # population = IndividualEP.elitism_selection(population, offspring)
        # tournament selection by pooling the parents and the offspring
        population = IndividualEP.tournament_selection(population, offspring)
        
    return population[0]


def es_1plus1(population_size, D, evaluatorFunction, x_min=-30, x_max=30):
    """The 1+1 evolution strategy."""
    population = [IndividialES(D, evaluatorFunction, x_min, x_max) for _ in range(population_size)]
    for _ in range(MAX_GENS):
        offspring = []
        for parent in population:
            kid = IndividialES.make_offspring(parent)
            offspring.append(IndividialES.select_best(parent, kid))
        population = offspring
    return population[0]
            

In [23]:
# due to the 2 algorithms are stochastic algorithms, we need to run it multiple times to get the best individuals, which is 30 times in this case

def run_algorthim(algorthim, experiment_times = 30 , D = 20 ,
                  evaluator_list = [Rosenbrock_function, Griewanks_function]) -> list:
    """Run the particular algorithm 30 times and return the best individuals.
    """
    # D = 20
    # evaluator_list = [Rosenbrock_function, Griewanks_function]
    print("-"* 100)
    print(f"Running the {algorthim.__name__} algorithm with D = {D} and experiment times = {experiment_times}...")
    
    best_individuals_dict = {evaluator:[] for evaluator in evaluator_list}
    # best_individuals_dict = {evaluator_list[0]: [], evaluator_list[1]: []}
    for evaluator in evaluator_list:
        # print(evaluator.__name__)
        for i in range(experiment_times):
            best_individual = algorthim(POPULATION_SIZE, D, evaluator)
            best_individuals_dict[evaluator].append(best_individual)
            
        
    # report the mean and the standard deviation of the best individuals, i.e. the mean and the standard deviation of the evaluator function values (i.e. fitness)
    for k,v in best_individuals_dict.items():
        print("-"*50)
        print(f"D = {D}, {k.__name__}")
        print(f"mean: {np.mean([x.fitness for x in v])}, std: {np.std([x.fitness for x in v])}")
        # print(f"Best individual: {min(v, key=lambda x: x.fitness)}")
        print(f"min (Best Individual Fitness): {min([x.fitness for x in v])}")

    return best_individuals_dict

In [8]:
fastEP_best_individual_dict_D20 = run_algorthim(fast_EP, D=20)


----------------------------------------------------------------------------------------------------
Running the fast_EP algorithm with D = 20 and experiment times = 30...
--------------------------------------------------
D = 20, Rosenbrock_function
mean: 158.43160867373098, std: 150.32652968940687
min (Best Individual Fitness): 18.547362941435914
--------------------------------------------------
D = 20, Griewanks_function
mean: 0.0675759763115388, std: 0.08382772652727391
min (Best Individual Fitness): 0.0008769633091648688


In [11]:
es_1plus1_best_individual_dict_D20 = run_algorthim(es_1plus1, D=20) #  , experiment_times=5)

----------------------------------------------------------------------------------------------------
Running the es_1plus1 algorithm with D = 20 and experiment times = 30...
--------------------------------------------------
D = 20, Rosenbrock_function
mean: 1258.5224452405187, std: 2878.3113830277402
min (Best Individual Fitness): 28.859184473671434
--------------------------------------------------
D = 20, Griewanks_function
mean: 0.033216029654704056, std: 0.033282135508776546
min (Best Individual Fitness): 0.0031430794953913876


for D = 50 requirement:

Then for D = 50, solve the Rosenbrock’s function using the same algorithm settings (repeat
30 times). Report the mean and standard deviation of your results. Compare the performance
of EP and DE/ES on the Rosenbrock’s function when D = 20 and D = 50. Analyze your
results, and draw you conclusions


In [24]:
fastEP_best_individual_dict_D50 = run_algorthim(fast_EP, D = 50, 
                                                evaluator_list = [Rosenbrock_function])


----------------------------------------------------------------------------------------------------
Running the fast_EP algorithm with D = 50 and experiment times = 30...
--------------------------------------------------
D = 50, Rosenbrock_function
mean: 6267860.798363612, std: 7829133.960825554
min (Best Individual Fitness): 283582.2587531093


In [25]:
es_1plus1_best_individual_dict_D50 = run_algorthim(es_1plus1, D = 50, 
                                                evaluator_list = [Rosenbrock_function])


----------------------------------------------------------------------------------------------------
Running the es_1plus1 algorithm with D = 50 and experiment times = 30...
--------------------------------------------------
D = 50, Rosenbrock_function
mean: 181725.31030583897, std: 109845.18553962128
min (Best Individual Fitness): 37992.92240439778
